In [1]:
pip install xml.etree.ElementTree

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement xml.etree.ElementTree (from versions: none)
ERROR: No matching distribution found for xml.etree.ElementTree

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\wpark11\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
pip install requests

## DICOM Standard Part 6

In [8]:
import requests
import xml.etree.ElementTree as ET

# URI for DICOM Standard Part 6
xml_uri = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part06/part06.xml'

# Parse the XML content
response = requests.get(xml_uri)
root = ET.fromstring(response.content)

In [9]:
# Find the child node with label="6"
for child in root:
    if child.attrib.get('label') == '6':
        selected_node = child
        break

In [42]:
import pandas as pd
from collections import defaultdict

# Get table 1 from Chapter 6
if selected_node is not None:
    grandchild_table = selected_node.find('.//{http://docbook.org/ns/docbook}table')
    if grandchild_table is not None:
        thead = grandchild_table.find('.//{http://docbook.org/ns/docbook}thead')
        tbody = grandchild_table.find('.//{http://docbook.org/ns/docbook}tbody')

        # Store column names
        column_names = []
        if thead is not None:
            for tr in thead.findall('.//{http://docbook.org/ns/docbook}tr'):
                thead_names = tr.findall('.//{http://docbook.org/ns/docbook}emphasis')
                if thead_names is not None:
                    for name in thead_names:
                        column_names.append(name.text.strip())

            # Store row values
            rows = tbody.findall('.//{http://docbook.org/ns/docbook}tr')
            rows_data = []

            # Loop through tbody to extract values
            for tr in rows:
                row_values = defaultdict(lambda: None)
                idx = 0
                for para in tr.findall('.//{http://docbook.org/ns/docbook}para'):
                    emphasis = para.find('.//{http://docbook.org/ns/docbook}emphasis')
                    if emphasis is not None and emphasis.text is not None:
                        row_values[column_names[idx]] = emphasis.text.strip()
                        idx += 1
                        if idx >= len(column_names):
                            break
                    else:
                        if para is not None and para.text is not None:
                            row_values[column_names[idx]] = para.text.strip()
                            idx += 1
                            if idx >= len(column_names):
                                break

                # Append to the table only if it has more than 3 values
                if sum(1 for value in row_values.values() if value is not None) > 3:
                    rows_data.append(row_values)

            # Save the output as a DataFrame
            df = pd.DataFrame(rows_data, columns=column_names)
else:
    print("Node with label='6' not found.")


In [53]:
column_names

['Tag', 'Name', 'Keyword', 'VR', 'VM']

In [43]:
df.head(10)

,Tag,Name,Keyword,VR,VM
0,"(0008,0001)",Length to End,Length​To​End,UL,1
1,"(0008,0005)",Specific Character Set,Specific​Character​Set,CS,1-n
2,"(0008,0006)",Language Code Sequence,Language​Code​Sequence,SQ,1
3,"(0008,0008)",Image Type,Image​Type,CS,2-n
4,"(0008,0010)",Recognition Code,Recognition​Code,SH,1
5,"(0008,0012)",Instance Creation Date,Instance​Creation​Date,DA,1
6,"(0008,0013)",Instance Creation Time,Instance​Creation​Time,TM,1
7,"(0008,0014)",Instance Creator UID,Instance​Creator​UID,UI,1
8,"(0008,0015)",Instance Coercion DateTime,Instance​Coercion​Date​Time,DT,1
9,"(0008,0016)",SOP Class UID,SOP​Class​UID,UI,1


In [20]:
included_VR = ['AT', 'CS', 'DA', 'DT', 'DS', 'FL', 'FD', 'IS', 'SL', 'SS', 'SV', 'TM', 'UL', 'US', 'UV']
df[df['VR'].isin(included_VR)].VR.value_counts() #CS is values; others are numbers 

VR
CS    799
DS    483
US    379
FD    309
FL    292
IS    239
UL     79
DT     62
DA     61
TM     55
AT     24
SS     18
SL     15
UV      6
SV      1
Name: count, dtype: int64

## DICOM Standard Part 16

In [3]:
import requests
import xml.etree.ElementTree as ET

# URI for DICOM Standard Part 6
xml_uri = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part16/part16.xml'

# Parse the XML content
response = requests.get(xml_uri)
root = ET.fromstring(response.content)

In [4]:
# Find the child node with label="6"
for child in root:
    if child.attrib.get('label') == 'B':
        selected_node = child         
        break
print(selected_node.attrib)

{'label': 'B', 'status': '1', '{http://www.w3.org/XML/1998/namespace}id': 'chapter_B'}


In [6]:
len(selected_node.findall('.//{http://docbook.org/ns/docbook}table'))

1347

In [39]:
import pandas as pd

# Get all tables from Chapter B
if selected_node is not None:
    table_elements = selected_node.findall('.//{http://docbook.org/ns/docbook}table')

    # Dictionary to store DataFrames
    tables_dict = {}

    # Iterate through each table
    for index, table_element in enumerate(table_elements):
        thead = table_element.find('.//{http://docbook.org/ns/docbook}thead')
        tbody = table_element.find('.//{http://docbook.org/ns/docbook}tbody')

        # Store column names
        column_names = []
        if thead is not None:
            # Store the column names for the first row
            for tr in thead.findall('.//{http://docbook.org/ns/docbook}tr'):
                for idx in range(len(tr)):
                    if tr[idx][0].text is not None:
                        column_names.append(tr[idx][0].text.strip())
        # print(f"Column names for Table {index + 1}: {column_names}")


        # Store row values
        rows_data = []
        if tbody is not None:
            # Iterate through each row in tbody
            for tr in tbody.findall('.//{http://docbook.org/ns/docbook}tr'):
                row_values = {}
                for idx, td in enumerate(tr.findall('.//{http://docbook.org/ns/docbook}td')):
                    if idx < len(column_names):
                        # Extract values from nested elements, such as links
                        value = ' '.join(text.strip() for text in td.itertext())
                        if value:
                            row_values[column_names[idx]] = value.strip()

                rows_data.append(row_values)

        # Save the output as a DataFrame and add to the dictionary
        df = pd.DataFrame(rows_data, columns=column_names)
        tables_dict[f'df_{index + 1}'] = df

else:
    print("Node with label='B' not found.")


## As functions

In [26]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict

def extract_DICOM_Part_6_table_data(xml_url, label):
    # Parse the XML content
    response = requests.get(xml_url)
    root = ET.fromstring(response.content)

    # Find the child node with the specified label
    selected_node = None
    for child in root:
        if child.attrib.get('label') == label:
            selected_node = child
            break

    # Get the first table from the selected node
    if selected_node is not None:
        grandchild_table = selected_node.find('.//{http://docbook.org/ns/docbook}table')
        if grandchild_table is not None:
            thead = grandchild_table.find('.//{http://docbook.org/ns/docbook}thead')
            tbody = grandchild_table.find('.//{http://docbook.org/ns/docbook}tbody')

            # Store column names
            column_names = []
            if thead is not None:
                for tr in thead.findall('.//{http://docbook.org/ns/docbook}tr'):
                    thead_names = tr.findall('.//{http://docbook.org/ns/docbook}emphasis')
                    if thead_names is not None:
                        for name in thead_names:
                            column_names.append(name.text.strip())

            # Store row values
            rows_data = []
            if tbody is not None:
                # Loop through tbody to extract values
                for tr in tbody.findall('.//{http://docbook.org/ns/docbook}tr'):
                    row_values = defaultdict(lambda: None)
                    idx = 0
                    for para in tr.findall('.//{http://docbook.org/ns/docbook}para'):
                        emphasis = para.find('.//{http://docbook.org/ns/docbook}emphasis')
                        if emphasis is not None and emphasis.text is not None:
                            row_values[column_names[idx]] = emphasis.text.strip()
                            idx += 1
                            if idx >= len(column_names):
                                break
                        else:
                            if para is not None and para.text is not None:
                                row_values[column_names[idx]] = para.text.strip()
                                idx += 1
                                if idx >= len(column_names):
                                    break

                    # Append to the table only if it has more than 3 values
                    if sum(1 for value in row_values.values() if value is not None) > 3:
                        rows_data.append(row_values)

                # Save the output as a DataFrame
                df = pd.DataFrame(rows_data, columns=column_names)

        else:
            print(f"No table found in node with label='{label}'.")

    else:
        print(f"Node with label='{label}' not found.")


In [27]:
# URI for DICOM Standard Part 6
xml_uri = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part06/part06.xml'

# Extract tables with label '6'
extract_DICOM_Part_6_table_data(xml_uri, '6')

In [38]:
df.head(5)

,Tag,Name,Keyword,VR,VM
0,"(0008,0001)",Length to End,Length​To​End,UL,1
1,"(0008,0005)",Specific Character Set,Specific​Character​Set,CS,1-n
2,"(0008,0006)",Language Code Sequence,Language​Code​Sequence,SQ,1
3,"(0008,0008)",Image Type,Image​Type,CS,2-n
4,"(0008,0010)",Recognition Code,Recognition​Code,SH,1


In [28]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

def extract_DICOM_Part_16_table_data(xml_url, label):
    # Parse the XML content
    response = requests.get(xml_url)
    root = ET.fromstring(response.content)

    # Find the child node with the specified label
    selected_node = None
    for child in root:
        if child.attrib.get('label') == label:
            selected_node = child
            break

    # Get all tables from the selected node
    if selected_node is not None:
        table_elements = selected_node.findall('.//{http://docbook.org/ns/docbook}table')

        # Dictionary to store DataFrames
        tables_dict = {}

        # Iterate through each table
        for index, table_element in enumerate(table_elements):
            thead = table_element.find('.//{http://docbook.org/ns/docbook}thead')
            tbody = table_element.find('.//{http://docbook.org/ns/docbook}tbody')

            # Store column names
            column_names = []
            if thead is not None:
                # Store the column names for the first row
                for tr in thead.findall('.//{http://docbook.org/ns/docbook}tr'):
                    for idx in range(len(tr)):
                        if tr[idx][0].text is not None:
                            column_names.append(tr[idx][0].text.strip())

            # Store row values
            rows_data = []
            if tbody is not None:
                # Iterate through each row in tbody
                for tr in tbody.findall('.//{http://docbook.org/ns/docbook}tr'):
                    row_values = {}
                    for idx, td in enumerate(tr.findall('.//{http://docbook.org/ns/docbook}td')):
                        if idx < len(column_names):
                            # Extract values from nested elements, such as links
                            value = ' '.join(text.strip() for text in td.itertext())
                            if value:
                                row_values[column_names[idx]] = value.strip()

                    rows_data.append(row_values)

            # Save the output as a DataFrame and add to the dictionary
            df = pd.DataFrame(rows_data, columns=column_names)
            tables_dict[f'df_{index + 1}'] = df

        return tables_dict

    else:
        print(f"Node with label='{label}' not found.")
        return None

In [29]:
# URI for DICOM Standard Part 16
xml_uri = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part16/part16.xml'

# Extract tables with label 'B'
result_dict = extract_DICOM_Part_16_table_data(xml_uri, 'B')

In [37]:
result_dict['df_1'].head(5)

,Coding Scheme Designator,Code Value,Code Meaning,SNOMED-RT ID,UMLS Concept Unique ID
0,SCT,24028007,Right,G-A100,C0205090
1,SCT,7771000,Left,G-A101,C0205091
2,SCT,51440002,Bilateral,G-A102,C0238767
3,SCT,66459002,Unilateral,G-A103,C0205092
4,SCT,49370004,Lateral,G-A104,C0205093
